In [ ]:
!pip freeze | grep scikit-learn

In [ ]:
!python -V

In [ ]:
import pickle
import pandas as pd
import numpy as np
import os

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    return df

In [ ]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [ ]:
std_dev = np.std(y_pred)
print(f'What is the standard deviation of the predicted duration for this dataset? {std_dev:.2f}')

In [ ]:

df['ride_id'] = f'{2023:04d}/{2:02d}_' + df.index.astype('str')

In [ ]:
df

In [ ]:
df_result = pd.DataFrame()
df_result['duration'] = df['duration']
df_result['ride_id'] = df['ride_id']
df_result

In [ ]:
df_result.to_parquet(
    'output.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [ ]:
len(df_result)

In [ ]:

file_path = './output.parquet'
file_size_bytes = os.path.getsize(file_path)

file_size_kb = file_size_bytes / 1024
file_size_mb = file_size_bytes / (1024 ** 2)
file_size_gb = file_size_bytes / (1024 ** 3)


print(f'Size of the output.parquet file is: {file_size_mb:.2f} MB')

Now let's turn the notebook into a script.

Which command you need to execute for that?

jupyter nbconvert --to script starter.ipynb
